In [1]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StringType, DoubleType, IntegerType
from pyspark import SparkConf, SparkContext

## Criando a [Spark Session](http://127.0.0.1:4040/)
Configurações padrão utilizadas no _spark-defaults.conf_ em $SPARK_HOME/conf/ 

In [2]:
#spark.stop()

spark = SparkSession \
    .builder \
    .appName("Teste Metastore") \
    .config("spark.sql.warehouse.dir", "s3a://warehouse/") \
    .config("spark.sql.catalogImplementation", "hive")\
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")\
    .config("spark.hadoop.fs.s3a.access.key", "admin")\
    .config("spark.hadoop.fs.s3a.secret.key", "admin123")\
    .config("spark.hadoop.fs.s3a.fast.upload", "true")\
    .config("spark.hadoop.fs.s3a.path.style.access", "true")\
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")\
    .config("spark.hadoop.javax.jdo.option.ConnectionDriverName", "org.postgresql.Driver")\
    .config("spark.hadoop.javax.jdo.option.ConnectionURL", "jdbc:postgresql://postgres:5432/metastore_db")\
    .config("spark.hadoop.javax.jdo.option.ConnectionUserName", "hive")\
    .config("spark.hadoop.javax.jdo.option.ConnectionPassword", "hive123")\
    .config("spark.hadoop.datanucleus.schema.autoCreateAll", "true")\
    .config("spark.hadoop.datanucleus.schema.autoCreateTables", "true")\
    .config("spark.hadoop.datanucleus.fixedDatastore", "false")\
    .config("spark.hadoop.hive.metastore.schema.verification", "false")\
    .config("spark.hadoop.hive.metastore.schema.verification.record.version", "false")\
    .config("spark.sql.hive.metastore.version", "2.3.9")\
    .config("spark.sql.hive.metastore.uris", "thrift://0.0.0.0:9083")\
    .config("spark.sql.hive.metastore.jars", "builtin")\
    .enableHiveSupport() \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/23 20:20:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/02/23 20:20:57 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
path = "s3a://not-a-s3-bucket/input/sample-data.csv"

schema = StructType() \
        .add("Row ID", IntegerType(), True) \
        .add("Order ID", StringType(), True) \
        .add("Order Date", StringType(), True) \
        .add("Ship Date", StringType(), True) \
        .add("Ship Mode", StringType(), True) \
        .add("Customer ID", StringType(), True) \
        .add("Customer Name", StringType(), True) \
        .add("Segment", StringType(), True) \
        .add("Country", StringType(), True) \
        .add("City", StringType(), True) \
        .add("State", StringType(), True) \
        .add("Postal Code", StringType(), True) \
        .add("Region", StringType(), True) \
        .add("Product ID", StringType(), True) \
        .add("Category", StringType(), True) \
        .add("Sub-Category", StringType(), True) \
        .add("Product Name", StringType(), True) \
        .add("Sales", DoubleType(), True) \
        .add("Quantity", IntegerType(), True) \
        .add("Discount", DoubleType(), True) \
        .add("Profit", DoubleType(), True)

df = spark.read.csv(path = path, schema = schema, header = True)
df.show()

+------+--------------+----------+----------+--------------+-----------+------------------+-----------+-------------+---------------+--------------+-----------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|Row ID|      Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|     Customer Name|    Segment|      Country|           City|         State|Postal Code| Region|     Product ID|       Category|Sub-Category|        Product Name|   Sales|Quantity|Discount|  Profit|
+------+--------------+----------+----------+--------------+-----------+------------------+-----------+-------------+---------------+--------------+-----------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|     1|CA-2016-152156| 11/8/2016|11/11/2016|  Second Class|   CG-12520|       Claire Gute|   Consumer|United States|      Henderson|      Kentucky|      42420|  South|FUR-BO-10001798|   

In [4]:
df.write.option("compression", "snappy")\
    .mode('overwrite')\
    .partitionBy('Region')\
    .format('parquet')\
    .saveAsTable('tb_sales_test')

AnalysisException: Can not create the managed table('`tb_sales_test`'). The associated location('s3a://warehouse/tb_sales_test') already exists.

In [ ]:
spark.sql("SHOW TABLES").show()

In [ ]:
spark.catalog.tableExists(tableName = "tb_sales_test")

In [ ]:
spark.sql("SHOW PARTITIONS default.tb_sales_test").show()

In [ ]:
spark.sql("SELECT * FROM default.tb_sales_test LIMIT 10").show()

In [ ]:
spark.sql("CREATE DATABASE IF NOT EXISTS delta")
spark.sql("SHOW DATABASES").show()

In [ ]:
new_cols = [x.replace(' ', '_') for x in df.columns]

df = df.toDF(*new_cols)

In [ ]:
df.printSchema()

In [ ]:
df.write.format("delta").mode("overwrite").partitionBy("Region").saveAsTable("delta.superstore_table")

In [ ]:
spark.catalog.listTables(dbName = 'delta')

In [ ]:
spark.read.table("delta.superstore_table").show(5)

In [ ]:
spark.stop()